In [ ]:
import plotly.io as pio
print(pio.renderers.default)

In [ ]:
import plotly.express

In [ ]:
import pandas

In [ ]:
trees = pandas.read_parquet("../subtrees-clean.parquet")
hash_counts = trees.groupby(
    ['package_name', 'package_version', 'hash'],
).agg(
    deps_count=('deps_count', 'first'),
    example_repo_path=('repo_path', 'first'),
    occurrences=('repo_path', 'count'),
)
version_counts = hash_counts.groupby(
    ['package_name', 'package_version'],
).agg(
    occurrences=('occurrences', 'sum'),
)


In [ ]:
1/0

In [ ]:
runs = pandas.read_parquet("../subtrees-clean.parquet")

In [ ]:
runs.head()

In [ ]:
hash_counts = runs.hash.value_counts().reset_index()

In [ ]:
hash_deps = runs[['hash', 'deps_count', 'package_name']].drop_duplicates(subset=['hash'])

In [ ]:
merged = pandas.merge(hash_counts, hash_deps, left_on='index', right_on='hash')

In [ ]:
merged = merged.drop('hash_y', 1).rename(columns={"hash_x": "tree_ocurrences", "deps_count": "tree_size"})

In [ ]:
merged.head()

In [ ]:
# plotly.express.scatter(merged, x='tree_ocurrences', y='tree_size', hover_name='package_name')

In [ ]:
def plot_this(df):
    plot_data = df
    fig = plotly.express.scatter(plot_data, x='tree_ocurrences', y='tree_size', hover_name='package_name', log_x=True, log_y=True)
    return fig.show()

# plot_this(merged)

## Analysis

the utility gained by building a package tree is proportional to the number of users (tree_ocurrences) * the size of the tree (tree_size).

    utility = tree_ocurrences * tree_size
    log(utility) = log(tree_ocurrences * tree_size)
    log(utility) = log(tree_ocurrences) + log(tree_size)

therefore, lines of constant utility are straight downwards-sloping lines in the above log plot (we want to build the packages that are furthest to the top-right on this plot, like `frame-benchmarking`, `mio` and `winapi`)

In practice, tree_ocurrences only says how many *GitHub Repos* are using particular configurations of the crates. We want to know how many *people* are using them. For this, we need to use the crates.io download data.